In [80]:
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import numpy as np
import torch
import csv
from bert_score import score
import clickhouse_driver


# Данные

In [81]:
questions = [
    "Что такое курс рубля и почему он меняется?",
    "Зачем Банк России перешел к режиму плавающего курса?",
    "Проводит ли Банк России операции с целью укрепить или ослабить рубль?"
]

In [82]:
import pandas as pd
test_data= pd.read_csv("test.csv") 
test_data["answer"] = test_data["answer"].str.replace("\n", "")

In [83]:
test_data['answer'][0]

'    Курс рубля к иностранной валюте - это стоимость одного рубля,     выраженная в единицах иностранной валюты.    Курс рубля меняется под влиянием спроса и предложения на     иностранную валюту на валютном рынке.    На курс рубля могут влиять:    - Объемы внешней торговли    - Импортные и экспортные цены    - Уровни инфляции и процентных ставок    - Темпы экономического роста    - Периоды нестабильности    - Изменения денежно-кредитной политики    '

# Пайплайн PCA для понижения размерности, 768 - слишком много

In [84]:
pca = Pipeline(steps=[
    ('mean', StandardScaler(with_mean=True, with_std=False)),
    ('pca', PCA(n_components=3, random_state=42)),
    ('std', StandardScaler(with_mean=True, with_std=True))
])

# ClickHouse

In [21]:
# Вывод существующей таблицы
client = clickhouse_driver.Client(host='localhost')
client.execute("SELECT * FROM embeddings")

[(0,
  [1.4124195575714111, -0.07120827585458755, -1250316.75],
  'https://www.cbr.ru/about_br/publ/ddkp/'),
 (1,
  [-0.6445415616035461, 1.258795976638794, -7689503.0],
  'https://www.cbr.ru/dkp/exchange_rate/'),
 (2,
  [-0.767877995967865, -1.1875869035720825, -6609879.0],
  'https://cbr.ru/oper_br/o_a/currency/')]

# Загружаем языковую модель и токенизатор

In [23]:
llm_dirname = '../nsu-ai/team_code/models/llm'


In [24]:
llm_model = AutoModelForCausalLM.from_pretrained(llm_dirname, torch_dtype=torch.float16, device_map={"":0}) # device_map требует установки Accelerate
tokenizer = AutoTokenizer.from_pretrained(llm_dirname)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Question

# Методы для получения индекса наиболее подходящего документа по запросу (можно переделать на несколько индексов (размер контекста позволяет)

In [49]:
def get_closest_document_idx(query: list) -> int:
    embedding = model.encode(query)
    pca_embedding = pca.transform(embedding)
    pca_embedding = np.squeeze(pca_embedding)
    query_vector = pca_embedding.tolist()
    results = client.execute("""
    SELECT id, embedding, link
    FROM embeddings
    ORDER BY L2Distance(embedding, {}) ASC
    LIMIT 1
    """.format(query_vector))
    
    return results[0][0]
    

In [74]:
def get_completion(query: str, model, tokenizer, documents=test_data, with_link=False) -> str:
    device = "cuda:0"
    document_idx = get_closest_document_idx([query])
    found_text = documents['answer'][document_idx]
    prompt_template = f"""[INST]Ты полезный русскоязычный помощник банка россии, который вежливо отвечает на вопросы. Ты знаешь что: {found_text} Вопрос: {query} Отвечай только то, в чем уверен. [/INST]
    """
    prompt = prompt_template.format(query=query)
    

    encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

    model_inputs = encodeds.to(device)

    generated_ids = model.generate(**model_inputs, max_new_tokens=500)
    decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
    if with_link:
        return (decoded[0] + ' Ответ основан на ' + documents['link'][document_idx], len(prompt))
    return (decoded[0], len(prompt))


# Тест

In [79]:
result, l = get_completion(query='Что такое курс рубля к иностранной валюте?', model=llm_model, tokenizer=tokenizer, documents=test_data, with_link=True)
result[l+1:]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'Курс рубля к иностранной валюте - это стоимость одного рубля, выраженная в единицах иностранной валюты. Ответ основан на https://www.cbr.ru/about_br/publ/ddkp/'

# Подсчет BERT F1 Score - https://github.com/Tiiiger/bert_score/

In [75]:
refs = test_data['answer'].values.tolist()

In [76]:
answers = []
for q in questions:
    result, l = get_completion(query=q, model=llm_model, tokenizer=tokenizer, documents=test_data)
    answers.append(' '.join(result[l+1:].split()).strip())

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [77]:
P, R, F1 = score(answers, refs, lang='en', verbose=True) # Precision, Recall, F1

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 0.16 seconds, 19.32 sentences/sec


In [78]:
print('На подготовленных вопросах, усредненный BERT F1 score: ', F1.mean())

На подготовленных вопросах, усредненный BERT F1 score:  tensor(0.9388)
